In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

In [ ]:
from firedrake import *
import numpy as np
import matplotlib.pyplot as plt

def solve_square(mesh_size:int, elem_type:str, k:int):
  mesh = RectangleMesh(mesh_size, mesh_size, 1, 1, -1, -1)
  x, y = SpatialCoordinate(mesh)
  n = FacetNormal(mesh)

  u_ex = exp(sin(x + y) - sin(y))
  p_ex = grad(u_ex)


  if elem_type == "BDM":
    V = FunctionSpace(mesh, "BDM", k + 1)
  else:
    # here RT_{k+1} means RT_k in our course
    V = FunctionSpace(mesh, "RT", k + 1)
  Q = FunctionSpace(mesh, "DG", k)

  W = V * Q

  p, u = split(TrialFunction(W))
  q, v = split(TestFunction(W))

  f = -div(p_ex)
  g = u_ex

  a = dot(p, q) * dx + div(p) * v * dx + div(q) * u * dx
  L = -f * v * dx + g * dot(q, n) * ds

  bcs = DirichletBC(W.sub(1), g, "on_boundary")

  w = Function(W)
  solve(a == L, w, bcs)
  ph, uh = split(w)
  pHdiv = norm(p_ex - ph, "Hdiv")
  pL2 = norm(p_ex - ph)
  uL2 = norm(u_ex - uh)
  return pHdiv, pL2, uL2

def test_problem1(mesh_sizes, ks):
  m = len(mesh_sizes)
  n = len(ks)
  pHdiv = np.zeros([m, n])
  pL2 = np.zeros([m, n])
  uL2 = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      pHdiv[i, j], pL2[i, j], uL2[i, j] \
        = solve_square(mesh_sizes[i], "BDM", ks[j])
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(pHdiv[-1, j] / pHdiv[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(pHdiv), label = labels)
  plt.title("Hdiv of p for BDM")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(Hdiv of p)")
  plt.legend()
  plt.savefig("./BHp.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(pL2[-1, j] / pL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(pL2), label = labels)
  plt.title("L2 of p for BDM")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of p)")
  plt.legend()
  plt.savefig("./BLp.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for BDM")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./BLu.png")
  plt.show()

  pHdiv = np.zeros([m, n])
  pL2 = np.zeros([m, n])
  uL2 = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      pHdiv[i, j], pL2[i, j], uL2[i, j] \
        = solve_square(mesh_sizes[i], "RT", ks[j])
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(pHdiv[-1, j] / pHdiv[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(pHdiv), label = labels)
  plt.title("Hdiv of p for RT")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(Hdiv of p)")
  plt.legend()
  plt.savefig("./RHp.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(pL2[-1, j] / pL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(pL2), label = labels)
  plt.title("L2 of p for RT")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of p)")
  plt.legend()
  plt.savefig("./RLp.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for RT")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./RLu.png")
  plt.show()

test_problem1([10, 20, 40, 60, 100, 150], range(4))

In [ ]:
from firedrake import *
import numpy as np
import matplotlib.pyplot as plt

def solve_square_post(mesh_size:int, elem_type:str, k:int):
  mesh = RectangleMesh(mesh_size, mesh_size, 1, 1, -1, -1)
  x, y = SpatialCoordinate(mesh)
  n = FacetNormal(mesh)

  u_ex = exp(sin(x + y) - sin(y))
  p_ex = grad(u_ex)


  if elem_type == "BDM":
    V = FunctionSpace(mesh, "BDM", k + 1)
  else:
    # here RT_{k+1} means RT_k in our course
    V = FunctionSpace(mesh, "RT", k + 1)
  Q = FunctionSpace(mesh, "DG", k)

  W = V * Q

  p, u = split(TrialFunction(W))
  q, v = split(TestFunction(W))

  f = -div(p_ex)
  g = u_ex

  a = dot(p, q) * dx + div(p) * v * dx + div(q) * u * dx
  L = -f * v * dx + g * dot(q, n) * ds

  bcs = DirichletBC(W.sub(1), g, "on_boundary")

  w = Function(W)
  solve(a == L, w, bcs)
  ph, uh = split(w)

  QN = FunctionSpace(mesh, "DG", k + 1)
  R = FunctionSpace(mesh, "DG", 0)
  WN = QN * R
  uN, r = TrialFunctions(WN)
  vN, s = TestFunctions(WN)
  aN = dot(grad(uN), grad(vN)) * dx + r * vN * dx + s * uN * dx
  LN = f * vN * dx + s * uh * dx + \
   2 * avg(dot(ph, n) * vN) * dS + dot(ph, n) * vN * ds
  wF = Function(WN)
  solve(aN == LN, wF)
  uF, _ = split(wF)

  uL2 = norm(u_ex - uh)
  uL2F = norm(u_ex - uF)
  return uL2, uL2F

def test_problem2(mesh_sizes, ks):
  m = len(mesh_sizes)
  n = len(ks)

  uL2 = np.zeros([m, n])
  uL2F = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      uL2[i, j], uL2F[i, j] = solve_square_post(mesh_sizes[i], "BDM", ks[j])
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for BDM")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./BuL2.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2F[-1, j] / uL2F[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2F), label = labels)
  plt.title("L2 of u for BDM after post processing")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./BuL2F.png")
  plt.show()

  uL2 = np.zeros([m, n])
  uL2F = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      uL2[i, j], uL2F[i, j] = solve_square_post(mesh_sizes[i], "RT", ks[j])
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for RT")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./RuL2.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2F[-1, j] / uL2F[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2F), label = labels)
  plt.title("L2 of u for RT after post processing")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./RuL2F.png")
  plt.show()

test_problem2([10, 20, 40, 100], range(4))

In [ ]:
from firedrake import *
import gmsh
import numpy as np
import matplotlib.pyplot as plt

def solve_beta(mesh_size:int, elem_type:str, k:int, beta:float):
  gmsh.initialize()
  gmsh.model.add("t1")
  h = 2 / mesh_size
  gmsh.model.geo.addPoint(0, 0, 0, h, 1)
  gmsh.model.geo.addPoint(1, 0, 0, h, 2)
  gmsh.model.geo.addPoint(1, 1, 0, h, 3)
  gmsh.model.geo.addPoint(-1, 1, 0, h, 4)
  if beta >= 0.8:
    gmsh.model.geo.addPoint(-1, np.tan((1 - 1 / beta) * np.pi), 0, h, 5)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 5, 4)
    gmsh.model.geo.addLine(5, 1, 5)
    gmsh.model.geo.addCurveLoop([1,2,3,4,5], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
    gmsh.write("t1.msh")
  elif beta >= 4 / 7:
    gmsh.model.geo.addPoint(-1, -1, 0, h, 5)
    gmsh.model.geo.addPoint(np.tan((1 / beta - 3 / 2) * np.pi), 0, h, -1, 6)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 5, 4)
    gmsh.model.geo.addLine(5, 6, 5)
    gmsh.model.geo.addLine(6, 1, 5)
    gmsh.model.geo.addCurveLoop([1,2,3,4,5,6], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
    gmsh.write("t1.msh")
  else:
    gmsh.model.geo.addPoint(-1, -1, 0, h, 5)
    gmsh.model.geo.addPoint(1, -1, 0, h, 6)
    gmsh.model.geo.addPoint(1, np.tan((1 / beta - 2) * np.pi), 0, h, 7)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 5, 4)
    gmsh.model.geo.addLine(5, 6, 5)
    gmsh.model.geo.addLine(6, 7, 6)
    gmsh.model.geo.addLine(7, 1, 7)
    gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
    gmsh.write("t1.msh")


  mesh = Mesh("t1.msh")

  x, y = SpatialCoordinate(mesh)
  n = FacetNormal(mesh)

  theta = conditional(atan2(y, x) < 0, atan2(y, x) + 2 * np.pi, atan2(y, x))
  u_ex = (x * x + y * y) ** (beta / 2) * sin(beta * theta)
  # u_ex = exp(y - x) * sin(x + y)
  p_ex = grad(u_ex)


  if elem_type == "BDM":
    V = FunctionSpace(mesh, "BDM", k + 1)
  else:
    # here RT_{k+1} means RT_k in our course
    V = FunctionSpace(mesh, "RT", k + 1)
  Q = FunctionSpace(mesh, "DG", k)

  W = V * Q
  # print(W.dim())

  p, u = split(TrialFunction(W))
  q, v = split(TestFunction(W))

  f = -div(p_ex)
  g = u_ex

  a = dot(p, q) * dx + div(p) * v * dx + div(q) * u * dx
  L = -f * v * dx + g * dot(q, n) * ds

  bcs = DirichletBC(W.sub(1), g, "on_boundary")

  w = Function(W)
  solve(a == L, w, bcs)
  ph, uh = split(w)
  pHdiv = norm(p_ex - ph, "Hdiv")
  pL2 = norm(p_ex - ph)
  uL2 = norm(u_ex - uh)
  return pHdiv, pL2, uL2

def test_problem3(mesh_sizes, ks, beta):
  m = len(mesh_sizes)
  n = len(ks)
  pHdiv = np.zeros([m, n])
  pL2 = np.zeros([m, n])
  uL2 = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      pHdiv[i, j], pL2[i, j], uL2[i, j] \
        = solve_beta(mesh_sizes[i], "BDM", ks[j], beta)
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(pHdiv[-1, j] / pHdiv[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(pHdiv), label = labels)
  plt.title("Hdiv of p for BDM")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(Hdiv of p)")
  plt.legend()
  plt.savefig("./BHpb.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(pL2[-1, j] / pL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(pL2), label = labels)
  plt.title("L2 of p for BDM")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of p)")
  plt.legend()
  plt.savefig("./BLpb.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for BDM")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./BLub.png")
  plt.show()

  pHdiv = np.zeros([m, n])
  pL2 = np.zeros([m, n])
  uL2 = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      pHdiv[i, j], pL2[i, j], uL2[i, j] \
        = solve_beta(mesh_sizes[i], "RT", ks[j], beta)
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(pHdiv[-1, j] / pHdiv[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(pHdiv), label = labels)
  plt.title("Hdiv of p for RT")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(Hdiv of p)")
  plt.legend()
  plt.savefig("./RHpb.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(pL2[-1, j] / pL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(pL2), label = labels)
  plt.title("L2 of p for RT")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of p)")
  plt.legend()
  plt.savefig("./RLpb.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for RT")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./RLub.png")
  plt.show()


test_problem3([20, 40, 80, 160], range(4), 0.85)

In [ ]:
from firedrake import *
import gmsh
import numpy as np
import matplotlib.pyplot as plt

def solve_beta_post(mesh_size:int, elem_type:str, k:int, beta:float):
  gmsh.initialize()
  gmsh.model.add("t1")
  h = 2 / mesh_size
  gmsh.model.geo.addPoint(0, 0, 0, h, 1)
  gmsh.model.geo.addPoint(1, 0, 0, h, 2)
  gmsh.model.geo.addPoint(1, 1, 0, h, 3)
  gmsh.model.geo.addPoint(-1, 1, 0, h, 4)
  if beta >= 0.8:
    gmsh.model.geo.addPoint(-1, np.tan((1 - 1 / beta) * np.pi), 0, h, 5)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 5, 4)
    gmsh.model.geo.addLine(5, 1, 5)
    gmsh.model.geo.addCurveLoop([1,2,3,4,5], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
    gmsh.write("t1.msh")
  elif beta >= 4 / 7:
    gmsh.model.geo.addPoint(-1, -1, 0, h, 5)
    gmsh.model.geo.addPoint(np.tan((1 / beta - 3 / 2) * np.pi), -1, 0, h, 6)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 5, 4)
    gmsh.model.geo.addLine(5, 6, 5)
    gmsh.model.geo.addLine(6, 1, 6)
    gmsh.model.geo.addCurveLoop([1,2,3,4,5,6], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
    gmsh.write("t1.msh")
  else:
    gmsh.model.geo.addPoint(-1, -1, 0, h, 5)
    gmsh.model.geo.addPoint(1, -1, 0, h, 6)
    gmsh.model.geo.addPoint(1, np.tan((1 / beta - 2) * np.pi), 0, h, 7)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 5, 4)
    gmsh.model.geo.addLine(5, 6, 5)
    gmsh.model.geo.addLine(6, 7, 6)
    gmsh.model.geo.addLine(7, 1, 7)
    gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
    gmsh.write("t1.msh")


  mesh = Mesh("t1.msh")

  x, y = SpatialCoordinate(mesh)
  n = FacetNormal(mesh)

  theta = conditional(atan2(y, x) < 0, atan2(y, x) + 2 * np.pi, atan2(y, x))
  u_ex = (x * x + y * y) ** (beta / 2) * sin(beta * theta)
  p_ex = grad(u_ex)


  if elem_type == "BDM":
    V = FunctionSpace(mesh, "BDM", k + 1)
  else:
    # here RT_{k+1} means RT_k in our course
    V = FunctionSpace(mesh, "RT", k + 1)
  Q = FunctionSpace(mesh, "DG", k)

  W = V * Q

  p, u = split(TrialFunction(W))
  q, v = split(TestFunction(W))

  f = -div(p_ex)
  g = u_ex

  a = dot(p, q) * dx + div(p) * v * dx + div(q) * u * dx
  L = -f * v * dx + g * dot(q, n) * ds

  bcs = DirichletBC(W.sub(1), g, "on_boundary")

  w = Function(W)
  solve(a == L, w, bcs)
  ph, uh = split(w)

  QN = FunctionSpace(mesh, "DG", k + 1)
  R = FunctionSpace(mesh, "DG", 0)
  WN = QN * R
  uN, r = TrialFunctions(WN)
  vN, s = TestFunctions(WN)
  aN = dot(grad(uN), grad(vN)) * dx + r * vN * dx + s * uN * dx
  LN = f * vN * dx + s * uh * dx + \
   2 * avg(dot(ph, n) * vN) * dS + dot(ph, n) * vN * ds
  wF = Function(WN)
  solve(aN == LN, wF)
  uF, _ = split(wF)

  uL2 = norm(u_ex - uh)
  uL2F = norm(u_ex - uF)
  return uL2, uL2F

def test_problem4(mesh_sizes, ks, beta):
  m = len(mesh_sizes)
  n = len(ks)

  uL2 = np.zeros([m, n])
  uL2F = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      uL2[i, j], uL2F[i, j] = solve_beta_post(mesh_sizes[i], "BDM", ks[j], beta)
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for BDM")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./BuL2b.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2F[-1, j] / uL2F[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2F), label = labels)
  plt.title("L2 of u for BDM after post processing")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./BuL2Fb.png")
  plt.show()

  uL2 = np.zeros([m, n])
  uL2F = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      uL2[i, j], uL2F[i, j] = solve_beta_post(mesh_sizes[i], "RT", ks[j], beta)
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for RT")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./RuL2b.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2F[-1, j] / uL2F[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.2f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2F), label = labels)
  plt.title("L2 of u for RT after post processing")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig("./RuL2Fb.png")
  plt.show()

test_problem4([20, 40, 80, 160], range(4), 0.67)

In [ ]:
from firedrake import *
import gmsh
import numpy as np
import matplotlib.pyplot as plt

def solve_betam_post(mesh_size:int, elem_type:str, k:int, beta:float):
  gmsh.initialize()
  gmsh.model.add("t1")
  h = 2 / mesh_size
  C = 0.2
  if beta >= 0.8:
    temp = np.tan((1 - 1 / beta) * np.pi)
    gmsh.model.geo.addPoint(C, 0, 0, h, 1)
    gmsh.model.geo.addPoint(1, 0, 0, h, 2)
    gmsh.model.geo.addPoint(1, 1, 0, h, 3)
    gmsh.model.geo.addPoint(-1, 1, 0, h, 4)
    gmsh.model.geo.addPoint(-1, temp, 0, h, 5)
    gmsh.model.geo.addPoint(-C, temp * C, 0, h, 6)
    gmsh.model.geo.addPoint(-C, C, 0, h, 7)
    gmsh.model.geo.addPoint(C, C, 0, h, 8)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 5, 4)
    gmsh.model.geo.addLine(5, 6, 5)
    gmsh.model.geo.addLine(6, 7, 6)
    gmsh.model.geo.addLine(7, 8, 7)
    gmsh.model.geo.addLine(8, 1, 8)
    gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7,8], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
    gmsh.write("t1.msh")
  elif beta >= 4 / 7:
    temp = np.tan((1 / beta - 3 / 2) * np.pi)
    gmsh.model.geo.addPoint(C, 0, 0, h, 1)
    gmsh.model.geo.addPoint(1, 0, 0, h, 2)
    gmsh.model.geo.addPoint(1, 1, 0, h, 3)
    gmsh.model.geo.addPoint(-1, 1, 0, h, 4)
    gmsh.model.geo.addPoint(-1, -1, 0, h, 5)
    gmsh.model.geo.addPoint(temp, -1, 0, h, 6)
    gmsh.model.geo.addPoint(temp * C, -C, 0, h, 7)
    gmsh.model.geo.addPoint(-C, -C, 0, h, 8)
    gmsh.model.geo.addPoint(-C, C, 0, h, 9)
    gmsh.model.geo.addPoint(C, C, 0, h, 10)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 5, 4)
    gmsh.model.geo.addLine(5, 6, 5)
    gmsh.model.geo.addLine(6, 7, 6)
    gmsh.model.geo.addLine(7, 8, 7)
    gmsh.model.geo.addLine(8, 9, 8)
    gmsh.model.geo.addLine(9, 10, 9)
    gmsh.model.geo.addLine(10, 1, 10)
    gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7,8,9,10], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
    gmsh.write("t1.msh")
  else:
    temp = np.tan((1 / beta - 2) * np.pi)
    gmsh.model.geo.addPoint(C, 0, 0, h, 1)
    gmsh.model.geo.addPoint(1, 0, 0, h, 2)
    gmsh.model.geo.addPoint(1, 1, 0, h, 3)
    gmsh.model.geo.addPoint(-1, 1, 0, h, 4)
    gmsh.model.geo.addPoint(-1, -1, 0, h, 5)
    gmsh.model.geo.addPoint(1, -1, 0, h, 6)
    gmsh.model.geo.addPoint(1, temp, 0, h, 7)
    gmsh.model.geo.addPoint(C, temp * C, 0, h, 8)
    gmsh.model.geo.addPoint(C, -C, 0, h, 9)
    gmsh.model.geo.addPoint(-C, -C, 0, h, 10)
    gmsh.model.geo.addPoint(-C, C, 0, h, 11)
    gmsh.model.geo.addPoint(C, C, 0, h, 12)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 5, 4)
    gmsh.model.geo.addLine(5, 6, 5)
    gmsh.model.geo.addLine(6, 7, 6)
    gmsh.model.geo.addLine(7, 8, 7)
    gmsh.model.geo.addLine(8, 9, 8)
    gmsh.model.geo.addLine(9, 10, 9)
    gmsh.model.geo.addLine(10, 11, 10)
    gmsh.model.geo.addLine(11, 12, 11)
    gmsh.model.geo.addLine(12, 1, 12)
    gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7,8,9,10,11,12], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
    gmsh.write("t1.msh")

  mesh = Mesh("t1.msh")

  x, y = SpatialCoordinate(mesh)
  n = FacetNormal(mesh)

  theta = conditional(atan2(y, x) < 0, atan2(y, x) + 2 * np.pi, atan2(y, x))
  u_ex = (x * x + y * y) ** (beta / 2) * sin(beta * theta)
  p_ex = grad(u_ex)


  if elem_type == "BDM":
    V = FunctionSpace(mesh, "BDM", k + 1)
  else:
    # here RT_{k+1} means RT_k in our course
    V = FunctionSpace(mesh, "RT", k + 1)
  Q = FunctionSpace(mesh, "DG", k)

  W = V * Q

  p, u = split(TrialFunction(W))
  q, v = split(TestFunction(W))

  f = -div(p_ex)
  g = u_ex

  a = dot(p, q) * dx + div(p) * v * dx + div(q) * u * dx
  L = -f * v * dx + g * dot(q, n) * ds

  bcs = DirichletBC(W.sub(1), g, "on_boundary")

  w = Function(W)
  solve(a == L, w, bcs)
  ph, uh = split(w)

  QN = FunctionSpace(mesh, "DG", k + 1)
  R = FunctionSpace(mesh, "DG", 0)
  WN = QN * R
  uN, r = TrialFunctions(WN)
  vN, s = TestFunctions(WN)
  aN = dot(grad(uN), grad(vN)) * dx + r * vN * dx + s * uN * dx
  LN = f * vN * dx + s * uh * dx + \
   2 * avg(dot(ph, n) * vN) * dS + dot(ph, n) * vN * ds
  wF = Function(WN)
  solve(aN == LN, wF)
  uF, _ = split(wF)

  uL2 = norm(u_ex - uh)
  uL2F = norm(u_ex - uF)
  return uL2, uL2F

def test_problem5(mesh_sizes, ks, beta):
  m = len(mesh_sizes)
  n = len(ks)

  uL2 = np.zeros([m, n])
  uL2F = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      uL2[i, j], uL2F[i, j] = solve_betam_post(mesh_sizes[i], "BDM", ks[j], beta)
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.3f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for BDM")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig(f"./BuL2b_{beta:.2f}.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2F[-1, j] / uL2F[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.3f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2F), label = labels)
  plt.title("L2 of u for BDM after post processing")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig(f"./BuL2Fb_{beta:.2f}.png")
  plt.show()

  uL2 = np.zeros([m, n])
  uL2F = np.zeros([m, n])
  for i in range(m):
    for j in range(n):
      uL2[i, j], uL2F[i, j] = solve_betam_post(mesh_sizes[i], "RT", ks[j], beta)
      print(".", end = "")
    print("")

  labels = []
  for j in range(n):
    ord = -(np.log(uL2[-1, j] / uL2[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.3f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2), label = labels)
  plt.title("L2 of u for RT")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig(f"./RuL2b_{beta:.2f}.png")
  plt.show()

  labels = []
  for j in range(n):
    ord = -(np.log(uL2F[-1, j] / uL2F[0, j])) \
     / (np.log(mesh_sizes[-1] / mesh_sizes[0]))
    labels.append(f"k = {ks[j]}, ord = {ord:.3f}")

  plt.plot(np.log(np.array(mesh_sizes)), np.log(uL2F), label = labels)
  plt.title("L2 of u for RT after post processing")
  plt.xlabel("log(mesh_size)")
  plt.ylabel("log(L2 of u)")
  plt.legend()
  plt.savefig(f"./RuL2Fb_{beta:.2f}.png")
  plt.show()

test_problem5([20, 40, 80, 160], range(4), 0.55)